In [3]:
from pathlib import Path
import os

# Establece la raíz del proyecto manualmente
project_root = Path("F:/JCMDataCenter/Cursos/Evolve Academy/Data Scientist IA/Futpeak") # sobremesa
#project_root = Path("C:/Users/juanm/Desktop/FUTPEAK/Futpeak") # portátil

# Cambia el directorio de trabajo actual a esa raíz
os.chdir(project_root)

print("📁 Directorio de trabajo actual:", Path.cwd())


📁 Directorio de trabajo actual: F:\JCMDataCenter\Cursos\Evolve Academy\Data Scientist IA\Futpeak


In [4]:
import pandas as pd
import os

# Crear carpeta si no existe
os.makedirs('data/processed', exist_ok=True)

# Cargar el archivo de metadata
metadata_df = pd.read_csv('data/processed/cleaned_metadata.csv')

# Filtrar registros donde Gender sea exactamente 'male' (ignorando mayúsculas)
male_metadata_df = metadata_df[metadata_df['Gender'].str.lower() == 'male']

# Guardar el nuevo CSV en la ruta especificada
output_path = 'data/processed/male_only_metadata.csv'
male_metadata_df.to_csv(output_path, index=False)

# Confirmar cuántos registros se guardaron
print(f"Registros con género masculino guardados: {len(male_metadata_df)}")
print(f"Archivo guardado en: {output_path}")


Registros con género masculino guardados: 3344
Archivo guardado en: data/processed/male_only_metadata.csv


In [5]:
import pandas as pd
import os

# Crear carpeta de salida si no existe
os.makedirs('data/processed', exist_ok=True)

# Cargar los archivos
metadata_df = pd.read_csv('data/processed/cleaned_metadata.csv')
matchlogs_df = pd.read_csv('data/processed/cleaned_matchlogs.csv')

# Filtrar metadata por género masculino
male_metadata_df = metadata_df[metadata_df['Gender'].str.lower() == 'male']
male_ids = set(male_metadata_df['Player_ID'])

# Filtrar matchlogs por esos jugadores
filtered_matchlogs_df = matchlogs_df[matchlogs_df['Player_ID'].isin(male_ids)]

# Eliminar columnas duplicadas del matchlogs que vendrán del metadata
filtered_matchlogs_df = filtered_matchlogs_df.drop(columns=[col for col in ['Player_name', 'Position'] if col in filtered_matchlogs_df.columns])

# Merge con metadata para agregar info del jugador
merged_df = filtered_matchlogs_df.merge(
    male_metadata_df[['Player_ID', 'Player_name', 'Full_name', 'Birth_date', 'Age', 'Position', 'Footed', 'Birth_place', 'Nationality', 'Club']],
    on='Player_ID',
    how='left'
)

# Reordenar columnas
initial_cols = ['Player_name', 'Player_ID']
merged_df = merged_df[[col for col in initial_cols if col in merged_df.columns] + [col for col in merged_df.columns if col not in initial_cols]]

# Guardar el CSV final
output_path = 'data/processed/matchlogs_male_players.csv'
merged_df.to_csv(output_path, index=False)

# Confirmación
print(f"✅ CSV guardado con {len(merged_df)} registros y {len(merged_df.columns)} columnas en: {output_path}")




✅ CSV guardado con 574412 registros y 53 columnas en: data/processed/matchlogs_male_players.csv


In [6]:
print(f"Filas antes del merge: {len(filtered_matchlogs_df)}")
print(f"Filas después del merge: {len(merged_df)}")
print("Columnas nuevas añadidas:", set(merged_df.columns) - set(filtered_matchlogs_df.columns))


Filas antes del merge: 574412
Filas después del merge: 574412
Columnas nuevas añadidas: {'Position', 'Footed', 'Nationality', 'Age', 'Full_name', 'Club', 'Birth_date', 'Birth_place', 'Player_name'}


In [7]:
# 📊 1. Jugadores con más partidos registrados

merged_df.groupby(['Player_ID', 'Player_name']).size().sort_values(ascending=False).head(10)


Player_ID  Player_name      
b3e42951   Franco Armani        1136
162efffd   Rodrigo De Paul      1062
81442ecb   Marcos Acuna         1008
cf5c3ef1   Carlos Izquierdoz     992
01eb744d   Angel Correa          980
d8f11068   Nahuel Guzman         964
97a17c0b   Ignacio Fernandez     958
875ac6e9   Diego Churin          896
b6160ad0   Matias Dituro         892
1edc043c   Federico Carrizo      846
dtype: int64

In [8]:
# 📊 2. Total de minutos jugados por jugador (útil para validar volumen de datos por jugador)

merged_df['Minutes'] = pd.to_numeric(merged_df['Minutes'], errors='coerce')
merged_df.groupby(['Player_ID', 'Player_name'])['Minutes'].sum().sort_values(ascending=False).head(10)


Player_ID  Player_name      
b3e42951   Franco Armani        102620.0
d8f11068   Nahuel Guzman         86602.0
cf5c3ef1   Carlos Izquierdoz     86002.0
b6160ad0   Matias Dituro         80278.0
775d365e   Esteban Andrada       76520.0
81442ecb   Marcos Acuna          73746.0
162efffd   Rodrigo De Paul       72452.0
39fb428f   Victor Cuesta         67722.0
97a17c0b   Ignacio Fernandez     67022.0
82549f54   Fabricio Bustos       66034.0
Name: Minutes, dtype: float64

In [9]:
# 📊 3. Jugadores con mayor número de temporadas únicas

merged_df.groupby(['Player_ID', 'Player_name'])['Seasons'].nunique().sort_values(ascending=False).head(10)


Player_ID  Player_name               
476d6fbe   Carlos Auzqui                 17
3ca6106c   Augusto Solari                17
b14f755d   Ivan Marcone                  17
635c9552   Ezequiel Cerutti              16
24377e64   Guillermo Matias Fernandez    16
5887a70d   Nery Dominguez                16
b4519dd4   Nicolas Blandi                16
14cc32a6   Rafael Marcelo Delgado        16
cf5c3ef1   Carlos Izquierdoz             16
97a17c0b   Ignacio Fernandez             16
Name: Seasons, dtype: int64

In [10]:
# Paso 1: obtener jugadores únicos con su país
unique_players = merged_df[['Player_ID', 'Nationality']].drop_duplicates()

# Paso 2: contar jugadores por país
top_nationalities = unique_players['Nationality'].value_counts().head(10)
print(top_nationalities)



Nationality
England        541
Germany        462
Argentina      457
Spain          296
France         261
Brazil         254
Italy          248
Netherlands    103
Belgium         89
Portugal        38
Name: count, dtype: int64


In [11]:
merged_df['Goals'] = pd.to_numeric(merged_df['Goals'], errors='coerce')
top_goals = merged_df.groupby(['Player_ID', 'Player_name'])['Goals'].sum().sort_values(ascending=False).head(10)
print(top_goals)


Player_ID  Player_name         
9ccf4e14   German Cano             66.0
875ac6e9   Diego Churin            64.0
1b9399bc   Bernardo Cuesta         64.0
7fa737c4   Ezequiel Barco          62.0
42fd9c7f   Kylian Mbappe           59.0
fc2c5692   Juan Ignacio Dinenno    54.0
05341453   Cristian Guanca         54.0
84d35e44   Julio Furch             50.0
0cab947b   Nicolas Ibanez          48.0
834d0be5   Lucas Alario            46.0
Name: Goals, dtype: float64


In [28]:
import pandas as pd

# Cargar los archivos
raw_df = pd.read_csv("data/raw/raw_matchlogs_ordered.csv", dtype=str, low_memory=False)
cleaned_df = pd.read_csv("data/processed/cleaned_matchlogs.csv", dtype=str)

# Buscar filas con ID de Mbappé en cualquier columna del raw
mbappe_raw_rows = raw_df[raw_df.apply(lambda row: row.astype(str).str.contains("42fd9c7f", na=False).any(), axis=1)]

# Buscar filas por ID en el cleaned
mbappe_cleaned_rows = cleaned_df[cleaned_df['Player_ID'] == '42fd9c7f']

# Convertir goles a numérico y sumar
mbappe_cleaned_rows['Goals'] = pd.to_numeric(mbappe_cleaned_rows['Goals'], errors='coerce')
total_goals = mbappe_cleaned_rows['Goals'].sum()

# Resultados
print(f"▶️ Filas en raw con ID: {len(mbappe_raw_rows)}")
print(f"✅ Filas en cleaned con ID: {len(mbappe_cleaned_rows)}")
print(f"⚽ Goles totales en cleaned: {int(total_goals)}")




▶️ Filas en raw con ID: 674
✅ Filas en cleaned con ID: 507
⚽ Goles totales en cleaned: 59


C:\Users\juanm\AppData\Local\Temp\ipykernel_38192\2012172950.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mbappe_cleaned_rows['Goals'] = pd.to_numeric(mbappe_cleaned_rows['Goals'], errors='coerce')
